# Import Libraries

In [1]:
import re
import time
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
class StackOverflow():    
  
    def __init__():
        pass

    def extract_data_from_url(url):
        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        data = StackOverflow.parse_tagged_page(soup)
        return data

    def clean_scraped_data(text , keyname=None):
        if keyname == 'votes':
            return text.replace('\nvotes', '')
        elif keyname == 'answer':
            return text.replace('answers', '')
        return text
    
    def get_id(q):
        
        pattern = 'href="/questions/([^"]+)[\/]'
        q_id = re.search(pattern,q).group(1)
        return q_id
    
    
    def parse_tagged_page(soup):
        question_summaries = soup.select('.question-summary')
        key_names = ['question', 'votes', 'tags','answer']
        classes_needed = ['.question-hyperlink', '.vote', '.tags', '.status']
        datas = []
        for q_el in question_summaries:
            question_data = {}
            for i, _class in enumerate(classes_needed):
                sub_el = q_el.select(_class)[0]
                keyname = key_names[i]
                question_data[keyname] = StackOverflow.clean_scraped_data(sub_el.text.strip(), keyname=keyname)
                if _class == '.question-hyperlink':
                    question_data['id'] = StackOverflow.get_id(str(sub_el))
            datas.append(question_data)
            
        return datas
    
    
    def scrape_data(tag, query_filter, max_pages, pagesize):
        base_url = 'https://stats.stackexchange.com/questions/tagged/'
        datas = list()
        for p in range(max_pages):
            page_num = p + 1
            url = f'{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}'
            datas += StackOverflow.extract_data_from_url(url)
            time.sleep(0.5)
        return datas

In [21]:
parameter_url = {'tag' :'python',
'query_filter' :'votes',
'max_pages' :4,
'pagesize' :50}

In [22]:
scrape_data = StackOverflow.scrape_data(**parameter_url)

In [23]:
dfStackOverFlor = pd.DataFrame(scrape_data, columns=scrape_data[0].keys())

In [24]:
dfStackOverFlor.head()

,question,id,votes,tags,answer
0,Python as a statistics workbench,1595,369,r spss stata python,26
1,What is batch size in neural network?,153531,224,neural-networks python terminology keras,5
2,What is an embedding layer in a neural network?,182775,104,machine-learning neural-networks python word-e...,2
3,"What loss function for multi-class, multi-labe...",207794,84,neural-networks python loss-functions keras cr...,6
4,What algorithm should I use to detect anomalie...,152644,75,machine-learning time-series python computatio...,9


In [27]:
bagOfWords = {}
for row in dfStackOverFlor['tags'].apply(lambda row: row.split()):
    for i in row:
        bagOfWords[i] = bagOfWords.get(i, 0) + 1 

In [28]:
bagOfWords

{'r': 25,
 'spss': 2,
 'stata': 1,
 'python': 200,
 'neural-networks': 12,
 'terminology': 1,
 'keras': 6,
 'machine-learning': 38,
 'word-embeddings': 1,
 'loss-functions': 1,
 'cross-entropy': 1,
 'time-series': 17,
 'computational-statistics': 3,
 'anomaly-detection': 5,
 'cross-validation': 5,
 'scikit-learn': 46,
 'software': 4,
 'svm': 5,
 'feature-selection': 6,
 'statsmodels': 7,
 'pandas': 6,
 'deep-learning': 4,
 'survival': 1,
 'mortality': 1,
 'regression': 15,
 'logistic': 8,
 'cart': 4,
 'accuracy': 1,
 'precision-recall': 1,
 'curve-fitting': 2,
 'predictive-models': 2,
 'mape': 1,
 'pca': 10,
 'tensorflow': 3,
 'autoencoders': 1,
 'outliers': 2,
 'change-point': 3,
 'classification': 7,
 'conditional-random-field': 1,
 'libsvm': 1,
 'c++': 1,
 'dimensionality-reduction': 6,
 'discriminant-analysis': 2,
 'svd': 5,
 'clustering': 8,
 'k-means': 4,
 'circular-statistics': 1,
 'multicollinearity': 1,
 'standard-error': 1,
 'regression-coefficients': 1,
 'multi-class': 1,
 '